In [1]:
from datetime import datetime
from os.path import exists as file_exists
import pandas as pd
from pprint import pprint as pp
import re
import requests
from time import sleep
import xmltodict
import Credentials      # Get API keys, etc.

apikey = Credentials.prod_api
baseurl = 'https://api-na.hosted.exlibrisgroup.com'
item_query = '/almaws/v1/bibs/{mms_id}/holdings/{holding_id}/items/{item_pid}?apikey={apikey}'

exported_csv = "FullItemList.csv"
filled_csv = "FilledEnumChron.csv"
err_log_txt = "log.txt"

### Run this cell if you want to use the Limited-list CSV
exported_csv = "LimitedList.csv"

#### Create original df

In [80]:
# Depending on how the file was exported, column names may or may have either spaces or underscores
df = pd.read_csv(exported_csv, converters={'Item_ID': str, 'Holdings_ID': str, 'MMS_ID': str, 'Item ID': str, 'Holdings ID': str, 'MMS ID': str})

#### Clean up, tweak & format df

In [29]:
# Remove spaces from column names
df.columns = [c.replace(' ', '_') for c in df.columns]
# Rename certain columns
df = df.rename(columns={'Permanent_Location': 'Location', 'Item_Policy': 'Policy', 'Material_Type': 'Material'})
# Strip leading/trailing space from Description
df.Description = df.Description.str.strip()
# Collapse multiple spaces within the Description
df.Description.replace(' +', ' ', regex=True, inplace=True)

In [30]:
# Add columns for the Enum/Chron fields
EC_fields = ['Enum_A', 'Enum_B', 'Chron_I', 'Chron_J']
df[EC_fields] = None

#### The function for getting info from Description to Enum/Chron fields:

In [31]:
def fill_and_extract(regex, these_fields):
    exp = re.compile(regex)
    for i, f in enumerate(these_fields):
        df[f] = df['Description'].str.extract(exp, expand=True)[i].fillna(df[f])

#### Call it repeatedly for each regex that you come up with:
**[Test your Regex here](https://regex101.com/)**

In [32]:
# Just volume & nothing else
fill_and_extract(r'^v\. ?(\d+)$', ['Enum_A'])

In [33]:
# Just issue & nothing else
fill_and_extract(r'^no\. ?(\d+)$', ['Enum_B'])

In [34]:
# Just year or range of years (post-18th-century) & nothing else
fill_and_extract(r'^((?:1[89]|20)\d{2}(?:-(?:1[89]|20)?\d{2})?)$', ['Chron_I'])

In [35]:
# Volume + year (or range of years)
fill_and_extract(r'^v\. ?(\d+) ((?:1[89]|20)\d{2}(?:[\-\/](?:1[89]|20)?\d{2})?)$', ['Enum_A', 'Chron_I'])

In [36]:
# Issue + year (or range of years)
fill_and_extract(r'^no\. ?(\d+) ((?:1[89]|20)\d{2}(?:[\-\/](?:1[89]|20)?\d{2})?)$', ['Enum_B', 'Chron_I'])

#### Then once all those replacements are done, pull filled-in records out to a new dataframe

In [37]:
# Create a dataframe to hold JUST records that get filled
filled = pd.DataFrame()
# Populate the new dataframe with any records that now have at least one Enum/Chron field filled
filled = df.dropna(subset=EC_fields, thresh=1)

#### Then apply the changes via the API and log filled items to the Filled CSV

In [41]:
records = len(filled)
c = 0
needs_header=not file_exists(filled_csv) # Apparently we're creating the file, so it needs a header

### Run this bit to see what got "filled" before hitting the API

for index, row in filled.fillna('').iterrows():
    c += 1
    print(c, ' / '.join([row['MMS_ID'], row['Holdings_ID'], row['Item_ID']]),
          str(row['Description']),
          ' | '.join(x or '' for x in [row['Enum_A'], row['Enum_B'], row['Chron_I'], row['Chron_J']]),
          sep="\t")


In [ ]:
with open(err_log_txt, 'a') as err_log:
    for index, row in filled.fillna('').iterrows():
        c += 1
        r = requests.get(''.join([baseurl,
                                  item_query.format(mms_id=str(row['MMS_ID']),
                                                    holding_id=str(row['Holdings_ID']),
                                                    item_pid=str(row['Item_ID']),
                                                    apikey=apikey)]))
        rdict = xmltodict.parse(r.text)
        if r.status_code == 429:  # Too many requests--daily limit
            print()
            print('Reached API request limit for today. Stopping execution.')
            print()            
            ## Drop this record & everything after from "filled"
            filled = filled.iloc[:c-1]
            break
        if r.status_code != 200:
            e = xmltodict.parse(r._content)
            # Log the error
            print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                  ' Error FETCHING item ', row['Item_ID'], ': (', r.status_code, ') ',
                  e['web_service_result']['errorList']['error']['errorMessage'],
                  sep='',
                  file=err_log)
            # Remove this item from the "filled" df
            filled = filled.drop([index])
            continue
        if (c % (records/100) < 1):
            print(int(100*c/records), '% complete', sep='')#, end='\r')
            sleep(5)
            
        # Merge derived values into the retrieved data (rdict)
        rdict['item']['item_data']['enumeration_a'] = str(row['Enum_A'])
        rdict['item']['item_data']['enumeration_b'] = str(row['Enum_B'])
        rdict['item']['item_data']['chronology_i'] = str(row['Chron_I'])
        rdict['item']['item_data']['chronology_j'] = str(row['Chron_J'])
        # Set an internal note, if there's an empty one available
        if ('Enum/Chron derived from Description' not in rdict['item']['item_data'].values()):
            if (not rdict['item']['item_data']['internal_note_1']):
                rdict['item']['item_data']['internal_note_1'] = 'Enum/Chron derived from Description'
            elif (not rdict['item']['item_data']['internal_note_2']):
                rdict['item']['item_data']['internal_note_2'] = 'Enum/Chron derived from Description'
            elif (not rdict['item']['item_data']['internal_note_3']):
                rdict['item']['item_data']['internal_note_3'] = 'Enum/Chron derived from Description'
            else: # Nbd, just log it
                print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), ' No internal note available for item MMS ID ',
                    str(row['MMS_ID']), sep="", file=err_log)
 
        # Push the altered record back into Alma
        pxml = xmltodict.unparse(rdict)
        p = requests.put(''.join([baseurl,
                                  item_query.format(mms_id=row['MMS_ID'],
                                                    holding_id=row['Holdings_ID'],
                                                    item_pid=row['Item_ID'],
                                                    apikey=apikey)]),
                         data=pxml.encode('utf-8'), headers={'Content-Type': 'application/xml'})
        if r.status_code == 429:  # Too many requests--daily limit
            print()
            print('Reached API request limit for today. Stopping execution.')
            print()            
            ## Drop this record & everything after from "filled"
            filled = filled.iloc[:c-1]
            break
        if p.status_code != 200:
            e = xmltodict.parse(p._content)
            # Log the error
            print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), ' Error UPDATING item ', row['Item_ID'], ': (', p.status_code, ') ',
                e['web_service_result']['errorList']['error']['errorMessage'],
                sep='', file=err_log)
            # Remove this item from the "filled" df
            filled = filled.drop([index])
            continue
        print(c, ' / '.join([row['MMS_ID'], row['Holdings_ID'], row['Item_ID']]),
              row['Description'],
              ' | '.join(x or '' for x in [row['Enum_A'], row['Enum_B'], row['Chron_I'], row['Chron_J']]),
              sep="\t")

        # Log it to the CSV
        #    Btw, the 'to_frame().T' transposes it, so it all goes in as a single comma-separated row
        row.to_frame().T.to_csv(filled_csv, mode='a', index=False, header=needs_header)
        needs_header = False # Henceforth

1	991003653479706388 / 2274276260006388 / 2374276220006388	v.13 2013	13 |  | 2013 | 
2	991003653479706388 / 2274276260006388 / 2374276230006388	v.12 2013	12 |  | 2013 | 
3	991003653479706388 / 2274276260006388 / 2374276240006388	v.11 2012	11 |  | 2012 | 
4	991003653479706388 / 2274276260006388 / 2374276250006388	v.10 2012	10 |  | 2012 | 
5	991002600689706388 / 2268835710006388 / 2368835530006388	v.19	19 |  |  | 
6	991002600689706388 / 2268835710006388 / 2368835540006388	v.18	18 |  |  | 
7	991002600689706388 / 2268835710006388 / 2368835550006388	v.17	17 |  |  | 
8	991002600689706388 / 2268835710006388 / 2368835560006388	v.16	16 |  |  | 
9	991002600689706388 / 2268835710006388 / 2368835570006388	v.15	15 |  |  | 
10	991002600689706388 / 2268835710006388 / 2368835580006388	v.14	14 |  |  | 
1% complete
11	991002600689706388 / 2268835710006388 / 2368835590006388	v.13	13 |  |  | 
12	991002600689706388 / 2268835710006388 / 2368835600006388	v.12	12 |  |  | 
13	991002600689706388 / 2268835710006

#### Purge filled rows from the original CSV

In [53]:
# Purge filled records from the original df
df = df.loc[~df['Item_ID'].isin(filled['Item_ID'])]

# Re-create the original CSV from that df
df.to_csv(exported_csv, index=False) # By default, will overwrite

# View df & filled

In [ ]:
display(df)
display(filled)

# <span style="color:#cc0000">Undo</span> changes to some records!

In [ ]:
undo_csv="Undo.csv"
df = pd.read_csv(undo_csv, converters={'Item_ID': str, 'Holdings_ID': str, 'MMS_ID': str, 'Item ID': str, 'Holdings ID': str, 'MMS ID': str, 'Enum_A': str, 'Enum_B': str, 'Chron_I': str, 'Chron_J': str})
# Remove spaces from column names
df.columns = [c.replace(' ', '_') for c in df.columns]
# Rename certain columns
df = df.rename(columns={'Permanent_Location': 'Location', 'Item_Policy': 'Policy', 'Material_Type': 'Material'})
records = len(df)
c = 0

for index, row in df.fillna('').iterrows():
    c += 1
    
    # Get the current record
    print(c, ' / '.join((row['MMS_ID'], row['Holdings_ID'], row['Item_ID'])), str(row['Description']), sep="\t")
    r = requests.get(''.join([baseurl,
                              item_query.format(mms_id=str(row['MMS_ID']),
                                                holding_id=str(row['Holdings_ID']),
                                                item_pid=str(row['Item_ID']),
                                                apikey=apikey)]))
    rdict = xmltodict.parse(r.text)
    if r.status_code != 200:
        e = xmltodict.parse(r._content)
        # Output the error
        print('Error FETCHING item ', row['Item_ID'], ': (', r.status_code, ') ',
            e['web_service_result']['errorList']['error']['errorMessage'],
            sep='')
        continue
    if (c % (records/100) < 1):
        print(int(100*c/records), '% complete', sep='')#, end='\r')
        sleep(5)

    # Merge derived values into the retrieved data (rdict)
    rdict['item']['item_data']['enumeration_a'] = \
        rdict['item']['item_data']['enumeration_b'] = \
        rdict['item']['item_data']['chronology_i'] = \
        rdict['item']['item_data']['chronology_j'] = None
    # Set an internal note, if there's an empty one available
    if (rdict['item']['item_data']['internal_note_1'] == 'Enum/Chron derived from Description'):
        rdict['item']['item_data']['internal_note_1'] = None
    if (rdict['item']['item_data']['internal_note_2'] == 'Enum/Chron derived from Description'):
        rdict['item']['item_data']['internal_note_2'] = None
    if (rdict['item']['item_data']['internal_note_3'] == 'Enum/Chron derived from Description'):
        rdict['item']['item_data']['internal_note_3'] = None

    # Push the altered record back into Alma
    pxml = xmltodict.unparse(rdict)
    p = requests.put(''.join([baseurl,
                              item_query.format(mms_id=row['MMS_ID'],
                                                holding_id=row['Holdings_ID'],
                                                item_pid=row['Item_ID'],
                                                apikey=apikey)]),
                     data=pxml.encode('utf-8'), headers={'Content-Type': 'application/xml'})
    if p.status_code != 200:
        e = xmltodict.parse(p._content)
        # Log the error
        print('Error UPDATING item ', row['Item_ID'], ': (', p.status_code, ') ',
            e['web_service_result']['errorList']['error']['errorMessage'],
            sep='')
        continue


# Testing stuff

In [127]:
desc = 'Spring 1885'
mmmRE = '(Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|June?|July?|Aug(?:ust)?|Sept?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?|Spr(?:ing)?|Sum(?:mer)?|Fall?|Aut(?:umn)?|Win(?:ter)?)'
yyyyRE = '(1[89]|20)\d\d'
exp = re.compile('^' + mmmRE + ' *' + yyyyRE + '$')

exp.match(desc)

In [13]:
undo_csv="Undo.csv"
df = pd.read_csv(undo_csv, converters={'Item_ID': str, 'Holdings_ID': str, 'MMS_ID': str, 'Item ID': str, 'Holdings ID': str, 'MMS ID': str, 'Enum_A': str, 'Enum_B': str, 'Chron_I': str, 'Chron_J': str})
# Remove spaces from column names
df.columns = [c.replace(' ', '_') for c in df.columns]
# Rename certain columns
df = df.rename(columns={'Permanent_Location': 'Location', 'Item_Policy': 'Policy', 'Material_Type': 'Material'})
records = len(df)
c = 0


In [27]:
df = df[df['Description'].str.match('^v. ?\d\d+')==True]

In [28]:
df

,Location,Material,Policy,Title,MMS_ID,Holdings_ID,Item_ID,Description,Enum_A,Enum_B,Chron_I,Chron_J
5963,2L Ref Gov Doc Index Table,Book,Non Circulating,"United States treaty index, 1776- /",991003653479706388,2274276260006388,2374276220006388,v.13 2013,3,,2013,
5964,2L Ref Gov Doc Index Table,Book,Non Circulating,"United States treaty index, 1776- /",991003653479706388,2274276260006388,2374276230006388,v.12 2013,2,,2013,
5965,2L Ref Gov Doc Index Table,Book,Non Circulating,"United States treaty index, 1776- /",991003653479706388,2274276260006388,2374276240006388,v.11 2012,1,,2012,
5966,2L Ref Gov Doc Index Table,Book,Non Circulating,"United States treaty index, 1776- /",991003653479706388,2274276260006388,2374276250006388,v.10 2012,0,,2012,
6083,2L Stacks,Book,Monograph,Catalogue of scientific papers,991002600689706388,2268835710006388,2368835530006388,v.19,9,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
15692,2L Stacks,Book,Monograph,The Medallion edition of the works of Joseph C...,991000246179706388,2266601050006388,2366600830006388,v.21,1,,,
15693,2L Stacks,Book,Monograph,The Medallion edition of the works of Joseph C...,991000246179706388,2266601050006388,2366600840006388,v.22,2,,,
15694,2L Stacks,Book,Monograph,The Medallion edition of the works of Joseph C...,991000246179706388,2266601050006388,2366600850006388,v.20,0,,,
15695,2L Stacks,Book,Monograph,The Medallion edition of the works of Joseph C...,991000246179706388,2266601050006388,2366600860006388,v.19,9,,,


In [24]:
print(c, ' / '.join([row['MMS_ID'], row['Holdings_ID'], row['Item_ID']]),
              row['Description'],
              ' | '.join(x or '' for x in [row['Enum_A'], row['Enum_B'], row['Chron_I'], row['Chron_J']]),
              sep="\t")

0	991001602639706388 / 2276187460006388 / 2376187390006388	1900	 |  | 1900 | 
